In [2]:
#import libraries

import pandas as pd
import numpy as np
import re


In [9]:
#loading the data

from google.colab import files
uploaded = files.upload()


Saving FINAL_PALM_OIL_MODEL.xlsx to FINAL_PALM_OIL_MODEL.xlsx


In [11]:
# Opening the Excel work book and Loading each sheet into its own dataframe

import pandas as pd

xls = pd.ExcelFile("FINAL_PALM_OIL_MODEL.xlsx")

state = pd.read_excel(xls, sheet_name="State_Profit_Model")
yield_bench = pd.read_excel(xls, sheet_name="Yield_Benchmarks")
cost_table = pd.read_excel(xls, sheet_name="Cost_Table")
price = pd.read_excel(xls, sheet_name="Price_Data")


In [12]:
#Cleaing the Cost Table

def parse_cost(s):
    nums = re.findall(r"\d[\d,]*", str(s))
    vals = [int(n.replace(",","")) for n in nums] if nums else []
    return np.mean(vals) if vals else 0

cost_table["Cost_val"] = cost_table["Cost_₦_per_ha"].apply(parse_cost)
base_cost = cost_table["Cost_val"].sum()

base_cost


np.float64(3647220.0)

In [13]:
# Merging the Yield Benchmarks Into State Table

df = state.merge(yield_bench, on="State", how="left")


In [14]:
# Computing for CPO yield

df["CPO_Yield_per_ha"] = df["Yield_FFB_tons_per_ha"] * 0.22


In [15]:
# input base operating cost

df["Base_Cost_per_ha"] = base_cost


In [16]:
# Extracting the 2025 Base Price Per MT

price_base = price[(price["Scenario"]=="Base") & (price["Year"]==2025)]["Price_N₦_per_MT"].iloc[0]
df["Price_per_MT"] = price_base


In [17]:
# Compute Profitability Metrics
# Profit per hectare

df["Profit_per_ha"] = (df["CPO_Yield_per_ha"] * df["Price_per_MT"]) - df["Base_Cost_per_ha"]


In [19]:
#calculating hectares needed to reach ₦1 Billion


df["Hectares_for_1B"] = 1_000_000_000 / df["Profit_per_ha"]

In [20]:
# Retun of Investment ROI

df["ROI"] = df["Profit_per_ha"] / df["Base_Cost_per_ha"]

In [21]:
#Payback Period (years)

df["Payback_Years"] = df["Base_Cost_per_ha"] / df["Profit_per_ha"]

In [22]:
#Exporting the Clean Final Sheet

df.to_excel("Final_Profitability_Model.xlsx", index=False)

In [23]:
df


,State,Avg_Annual_Rainfall_mm,Temperature_°C,Rainfall_Suitability_Index_(1–5),Land Cost per ha (₦),Suitable_Land_ha,Suitability_Score_(1–5),No_of_Mills,Avg_Distance_to_Mill_km,Cost_Adjustment_Factor,Base_Cost_per_ha,Price_per_MT,Yield_FFB_tons_per_ha,CPO_Yield_per_ha,Profit_per_ha,Hectares_for_1B,ROI,Payback_Years
0,Edo,2200,26,5,1200000,200000,5,14,10,0.95,3647220.0,2500000,15.0,3.30,4602780.0,217.260004,1.261997,0.792395
1,Cross River,2700,25,5,1150000,180000,5,9,15,0.97,3647220.0,2500000,14.0,3.08,4052780.0,246.744210,1.111197,0.899930
2,Akwa Ibom,2400,26,5,1100000,150000,4,8,20,1.00,3647220.0,2500000,13.0,2.86,3502780.0,285.487527,0.960397,1.041236
3,Delta,2000,27,4,1000000,120000,4,7,25,1.03,3647220.0,2500000,12.0,2.64,2952780.0,338.663903,0.809597,1.235182
4,Ondo,1800,27,3,1050000,90000,3,5,30,1.05,3647220.0,2500000,10.0,2.20,1852780.0,539.729488,0.507998,1.968512
5,Rivers,2400,26,4,1050000,110000,4,6,22,1.01,3647220.0,2500000,NaN,NaN,NaN,NaN,NaN,NaN
6,Bayelsa,2600,25,4,900000,80000,3,4,28,1.04,3647220.0,2500000,NaN,NaN,NaN,NaN,NaN,NaN
